In [15]:
def plot_corr_matrix(df, subset=[], details="All Features"):
    """Plots a correlation matrix of all the features. Can specify subset of features if desired"""

    if len(subset)>0:
        df = df[subset]
        
    corr = df.corr()
    #print("First 3 correlations: "+str(corr.iloc[0,0:3]))
    #cmap = sn.diverging_palette(255, 133, l=60, n=7, center="dark")
    
    # The values number here ar just for "nice" scaling
    xx=(df.shape[1]/3,(df.shape[1]/3)*.85 )
    fig=plt.figure(figsize=xx)
    ax = sn.heatmap(corr,cmap='RdBu_r',vmin=-1,vmax=1)

    ax.set_title("Correlation for "+details)

In [2]:
def move_target_to_end(df,target_label):
    
    """Moves the target colums to the end of the DataFrame for easier book-keeping
    and correlation matrix display"""
    
    cols = df.columns.tolist()
    
    p = cols.index(target_label)
    cols_new=cols[0:p]
    cols_new = cols_new + cols[p+1:]
    cols_new.append(cols[p])
    
    df=df[cols_new]
    return df

In [11]:
def select_top_features(df,target_label,n_features=10,exclude_labels=None):
    """ Select the n features most highly correlated with the specified 
    target aviables. NOTE: should only run this with df that has the target 
    still embedded; will return erroneous result if used e.g. with x"""
    
    # Set upper limit on number of features that can be returned
    if exclude_labels is not None:
        length_constraint = len(df.columns)-1-len(exclude_labels)
    else:
        length_constraint = len(df.columns)-1
    
    # if n_features is too high, cap it and print a warning
    if n_features>length_constraint:
        print("warning: number of features cannot exceed "+str(length_constraint)+
              "\nSelecting "+str(length_constraint)+"features.")
        n_features = length_constraint
    
    df=move_target_to_end(df,target_label)
    
    corr_vals_sorted=abs(df.corr()).iloc[-1].sort_values(ascending=False)
    #print(exclude_labels)
    high_corr_feat = list(corr_vals_sorted.index[1:n_features+1])
    
    # count the excluded labels
    c=0
    if exclude_labels is not None:
        for li in range(len(exclude_labels)):
            if exclude_labels[li] in high_corr_feat:
                high_corr_feat.remove(exclude_labels[li]) 
                c+=1
                high_corr_feat.append(corr_vals_sorted.index[n_features+c])
            
    #print(high_corr_feat)
    return high_corr_feat

In [ ]:
def lin_reg(x,y, features=[],do_scale=True, show_cm=False,normTF=True):
    
    if len(features)>0:
        #print(len(features)>0)
        x=x[features]
    
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=3)
    
    model = LinearRegression()
    if do_scale:
        regressor = Pipeline(steps = [('scaler',StandardScaler()),('model',model)])
    else:
        regressor = Pipeline(steps = [('model',model)])
        
    regressor.fit(x_train,y_train)
    y_predict = regressor.predict(x_val)
    
    scores = cross_val_score(regressor, x, y, cv=5, scoring='neg_mean_squared_error')
    print("RMSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    # Display some random validation vs predicted y pairs
    rand_samp = list(np.random.choice(range(len(y_val)),10,replace=False))
    y_top = np.array(y_val.iloc[rand_samp])
    # Rounding the predicted values for display purposes
    yp_top = np.array(y_predict[rand_samp]).round()
    
    Y = np.array([y_top,yp_top]).T
    print(Y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def dtree_reg(x,y, features=[],do_scale=True, show_cm=False,normTF=True):
    
    if len(features)>0:
        #print(len(features)>0)
        x=x[features]
    
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=3)
    
    model = DecisionTreeRegressor(random_state=3)
    if do_scale:
        regressor = Pipeline(steps = [('scaler',StandardScaler()),('model',model)])
    else:
        regressor = Pipeline(steps = [('model',model)])
        
    regressor.fit(x_train,y_train)
    y_predict = regressor.predict(x_val)
    
    scores = cross_val_score(regressor, x, y, cv=5, scoring='neg_mean_squared_error')
    print("RMSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    # Display some random validation vs predicted y pairs
    rand_samp = list(np.random.choice(range(len(y_val)),10,replace=False))
    y_top = np.array(y_val.iloc[rand_samp])
    # Rounding the predicted values for display purposes
    yp_top = np.array(y_predict[rand_samp]).round()
    
    Y = np.array([y_top,yp_top]).T
    print(Y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def randfor_reg(x,y, features=[],do_scale=True, show_cm=False,normTF=True):
    
    if len(features)>0:
        #print(len(features)>0)
        x=x[features]
    
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=3)
    
    model = RandomForestRegressor(random_state=3, n_estimators=100)
    if do_scale:
        regressor = Pipeline(steps = [('scaler',StandardScaler()),('model',model)])
    else:
        regressor = Pipeline(steps = [('model',model)])
        
    regressor.fit(x_train,y_train)
    y_predict = regressor.predict(x_val)
    
    scores = cross_val_score(regressor, x, y, cv=5, scoring='neg_mean_squared_error')
    print("RMSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    # Display some random validation vs predicted y pairs
    rand_samp = list(np.random.choice(range(len(y_val)),10,replace=False))
    y_top = np.array(y_val.iloc[rand_samp])
    # Rounding the predicted values for display purposes
    yp_top = np.array(y_predict[rand_samp]).round()
    
    Y = np.array([y_top,yp_top]).T
    print(Y)

In [ ]:
from sklearn.metrics import classification_report
def print_gridsearch_results(clf,x_valid, y_valid):
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_true, y_pred = y_test, clf.predict(X_test)
    y_pred = clf.predict(x_valid)
    print(classification_report(y_valid, y_pred))
    print()